<a href="https://colab.research.google.com/github/mitali-g/MachineLearning/blob/master/NEW_transfer_learning_chest_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import os, cv2, random
import pandas as pd
%matplotlib inline

In [ ]:
dataset = pd.read_csv("Chest_xray_Corona_Metadata.csv")
print(dataset.shape)

dataset = dataset[['X_ray_image_name', 'Label']]

# define train and test dataframe
train_dataset = dataset[0:5282]
test_dataset = dataset[5282:]

# shuffle values in train and test
from sklearn.utils import shuffle
train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

# define dataframe with train features
x_train_features = train_dataset['X_ray_image_name']

# define dataframe with train labels
y_train_labels = train_dataset['Label']

# define dataframe with test features
x_test_features = test_dataset['X_ray_image_name']

# define dataframe with test labels
y_test_labels = test_dataset['Label']

In [ ]:
# create y_train
y_train = []
for label in y_train_labels:
    if label == 'Pnemonia':
        y_train.append(1)
    elif label == 'Normal':
        y_train.append(0)
    else:
        print(label)
y_train = tf.keras.utils.to_categorical(y_train)

# create y_test
y_test = []
for label in y_test_labels:
    if label == 'Pnemonia':
        y_test.append(1)
    elif label == 'Normal':
        y_test.append(0)
    else:
        print(label)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# create x_train
x_train = []
train_path = ('train-color/')

for image in x_train_features:
    if ('.jpg' in image) == True:
        image = image[:image.rfind('.jpg')]
    elif ('.jpeg' in image) == True:
        image = image[:image.rfind('.jpeg')]
    elif ('.png' in image) == True:
        image = image[:image.rfind('.png')]
    image_path = train_path + image
    
    if os.path.isfile(image_path + '.png'):
        image = plt.imread(image_path + '.png')
        plt.grid(False)
        x_train.append(image)
len(x_train)

In [ ]:
# create x_test
x_test = []

test_path = ('test-color/')

for image in x_test_features:
  # image.rfind : removes '.jpeg'
    image_path = test_path + image[:image.rfind('.jpeg')]

    if os.path.isfile(image_path + '.png'):
        image = plt.imread(image_path + '.png')
        plt.grid(False)
#         plt.imshow(image)
#         plt.show()

        x_test.append(image)
len(x_test)

In [ ]:
# convert x_train, y_train, x_test, y_test to numpy arrays
x_train = np.array(x_train, np.float16) / 255.0
y_train = np.array(y_train)

print(len(x_train), len(y_train), x_train.shape, y_train.shape)

x_test = np.array(x_test, np.float16) / 255.0
y_test = np.array(y_test)
print(len(x_test), len(y_test), x_test.shape, y_test.shape)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
train_input = preprocess_input(x_train)
train_features = model.predict(train_input)
np.savez('vgg16_features_train', features=train_features)

In [ ]:
test_input = preprocess_input(x_train)
test_features = model.predict(test_input)
np.savez('vgg16_features_test', features=test_features)